In [ ]:
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "JSON",
    "ProgressMeter",
    "SHA"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))

In [ ]:
sra_dir = joinpath(data_dir, "SRA")

In [ ]:
import Printf
# Function to format file sizes in a human-readable format
function human_readable(size)
    if size < 1024
        return "$size Bytes"
    elseif size < 1048576
        return Printf.@sprintf("%.2f KiB", size / 1024)
    elseif size < 1073741824
        return Printf.@sprintf("%.2f MiB", size / 1048576)
    else
        return Printf.@sprintf("%.2f GiB", size / 1073741824)
    end
end

In [ ]:
# # Step 1: List Existing Files on Google Drive

# # We assume you have rclone installed and configured.
# # Note: You'll need to have system calls to rclone. Ensure `rclone` is in your PATH.

# # Run the rclone command to list files from Google Drive
# drive_file = "drive_file_list.json"
# if !isfile(drive_file)
#     println("run me")
#     # several minutes
#     @time run(pipeline(`/scg/apps/software/rclone/1.67.0/rclone lsjson --hash -R snyder-virome:viral-exposome/data/SRA`, drive_file))
# else
#     println("$(drive_file) already present")
# end

# # Load the data from the JSON file
# # quick 160seconds
# @time gdrive_files = filter(gf -> gf["IsDir"] == false, JSON.parsefile(drive_file))
# gdrive_files = convert(Vector{first(unique(typeof.(gdrive_files)))}, gdrive_files)
# gdrive_files = sort(gdrive_files, by=x->x["Size"])

# # Check and remove local files that match Google Drive files
# local_dir = sra_dir

# # Initialize counters for total files removed and total disk space reclaimed
# total_files_removed = 0
# total_space_freed = 0

# # Total files removed: 3626
# # Total space freed: 327.15 GiB

# ProgressMeter.@showprogress for gdrive_file in gdrive_files
#     file_path = gdrive_file["Path"]
#     local_file_path = joinpath(local_dir, file_path)
#     local_size = filesize(local_file_path)
#     if isfile(local_file_path)
#         # Calculate local file size and hashes
#         size_match = (local_size == gdrive_file["Size"])
#         # println()
#         # println("file_size = $(human_readable(local_size))" * "\t" * "file_path = $(file_path)")
        
#         # Compare sizes and hashes
#         if size_match
#             sha256 = SHA.bytes2hex(SHA.sha256(read(local_file_path)))
#             sha256_match = (sha256 == gdrive_file["Hashes"]["sha256"])
#             if sha256_match
#                 # Remove the local file
#                 rm(local_file_path)
#                 total_files_removed += 1
#                 total_space_freed += local_size
#                 # Print details
#                 # println("Removed file: $file_path")
#                 # println("Freed: $(human_readable(local_size))")
#             end
#         end
#     else
#         # println("$file_path has already been removed")
#     end
# end
# println("Total files removed: $total_files_removed")
# println("Total space freed: $(human_readable(total_space_freed))\n")

In [ ]:
# hold on this until we have deleted all of the above local files. Then just rclone copy again, repeat until all local files are deleted!

In [ ]:
# 267518 before cleaning up
# 43849 after
# 31194 left
# 75.346661 seconds (232.25 k allocations: 26.386 MiB, 0.11% compilation time)
# 41.443591 seconds (231.54 k allocations: 26.367 MiB, 0.20% compilation time)
# 28.985454 seconds (231.28 k allocations: 26.359 MiB, 0.31% compilation time)
# 80.912376 seconds (210.61 k allocations: 23.811 MiB, 0.10% compilation time)
@time local_files = readlines(`find $(sra_dir) -type f -print`)

In [ ]:
# function evaluate_file_path(file_path::String)
#     !isfile(file_path) && return
#     # Extensions to keep
#     keep_extensions = Set([
#         ".tsv", ".txt", ".gz", ".bam", ".sam", ".fasta", ".fna", ".fa", ".faa", ".png", 
#         ".json", ".css", ".pdf", ".gfa", ".html", ".jpg", ".gif", ".jpeg", ".svg", 
#         ".yaml", ".zip", ".fastg", ".gff3", ".js", ".gff", ".gb", ".genbank", ".gbk", 
#         ".krona", ".UniRef100", ".UniRef100_report", ".UniRef100_tophit_aln", 
#         ".UniRef100_tophit_report", ".UniRef50_report", ".UniRef50_tophit_aln", 
#         ".UniRef50_tophit_report", ".UniRef90_report", ".UniRef90_tophit_aln", ".UniRef90_tophit_report"
#     ])

#     # Extensions to remove
#     remove_extensions = Set([
#         ".info", ".source", ".log", ".index", ".lookup", ".ftr", ".clf", ".hmmtbl", 
#         ".taxwhm", ".npz", ".tax", ".ann", ".err", ".preext", ".pac", ".sg", 
#         ".counting", ".bdy", ".ftr", ".amb", ".bin", ".bwt", ".dbtype", ".lib", 
#         ".lib_info", ".list", ".partial", ".sa", ".sdbg_info", ".sh", ".split", 
#         ".splithmmtbl", ".swp", ".tfrec"
#     ])

#     # Split the file path into components
#     path_parts = split(file_path, "/")
#     # Extract the file name and extension
#     file_name = path_parts[end]
#     file_ext = splitext(file_name)[end]

#     # Flags
#     keep_flag = false
#     remove_flag = false
#     warning_flag = false

#     # Check for remove extensions and numeric extensions
#     if file_ext in remove_extensions || all(x -> isdigit(x), splitext(file_name)[2][2:end])
#         remove_flag = true
#     elseif any(x -> startswith(x, "."), path_parts) || any(x -> occursin(r"^tmp$"i, x), path_parts) || occursin("/megahit/intermediate_contigs/", file_path) || occursin("/virsorter/iter-0/", file_path)
#         remove_flag = true
#     elseif file_ext in keep_extensions
#         keep_flag = true
#     else
#         warning_flag = true
#     end

#     # Determine action based on flags
#     if keep_flag && !remove_flag && !warning_flag
#         # println("keeping: $file_path")
#     elseif remove_flag && !keep_flag && !warning_flag
#         # println("removing: $file_path")
#         rm(file_path)
#     else
#         println("warning: human review needed for $file_path")
#     end
# end

# # [1:2^7]
# for local_file in local_files
#     evaluate_file_path(local_file)
# end

In [ ]:
# prioritize the largest files (the most data rich ones!)
# 23 seconds
# 22.738189 seconds (1.78 M allocations: 350.219 MiB, 2.63% gc time, 1.80% compilation time)
# 527.604834 seconds (1.83 M allocations: 359.887 MiB, 0.04% gc time, 0.08% compilation time)!!!!!!
@time local_files = sort(local_files, by=x->filesize(x), rev=true)

In [ ]:
human_readable(filesize(first(local_files)))

In [ ]:
delay = 60
endpoints = ["snyder-virome-alt:", "snyder-virome-gmail:", "snyder-virome-lbl:", "snyder-virome:"]

for local_file in local_files
    !isfile(local_file) && continue
    attempt = 0
    success = false
    while attempt < 10 && !success
        for i in 1:length(endpoints)
            try
                fs = human_readable(filesize(local_file))
                # println("Uploading $(fs) file $(local_file) using endpoint $(endpoints[i])...")
                remote_file = replace(local_file, "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/" => endpoints[i])
                remote_dir = dirname(remote_file)
                run(`/scg/apps/software/rclone/1.67.0/rclone copy --drive-chunk-size 1G --drive-upload-cutoff 1T --tpslimit 1 --tpslimit-burst 0 --low-level-retries=1 --retries=1 --retries-sleep=1m --checkers 1 --transfers 1 --max-backlog 1 --drive-pacer-burst 1 --drive-pacer-min-sleep 1s $(local_file) $(remote_dir)`)
                success = true
                if i > 1
                    pushfirst!(endpoints, popat!(endpoints, i))
                end
                break # exit the for loop if successful
            catch e
                println("Error uploading\n$(local_file)\n$(e)\nSwitching endpoint and retrying.")
                if i == length(endpoints)
                    println("All endpoints failed, retrying in $(delay * (2 ^ attempt)) seconds.")
                    sleep(delay * (2 ^ attempt))
                    attempt += 1
                end
            end
        end
    end
    if success
        # Delete local file
        println("Successfully uploaded $(local_file). removing...")
        rm(local_file)
    else
        println("Failed to upload $(local_file) after $(attempt) attempts. Try again later?")
        break
    end
end

In [ ]:
# # delay = 120
# delay = 60
# for local_file in local_files
#     !isfile(local_file) && continue
#     attempt = 0
#     success = false
#     while attempt < 10 && !success
#         try
#             fs = human_readable(filesize(local_file))
#             println("Uploading $(fs) file $(local_file)...")
#             # run(`/scg/apps/software/rclone/1.67.0/rclone copy $(local_file) $(remote_dir)`)
#             remote_file = replace(local_file, "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/" => "snyder-virome:")
#             remote_dir = dirname(remote_file)
#             run(`/scg/apps/software/rclone/1.67.0/rclone copy --drive-chunk-size 1G --drive-upload-cutoff 1T --tpslimit 1 --tpslimit-burst 0 --low-level-retries=1 --retries=1 --retries-sleep=1m --checkers 1 --transfers 1 --max-backlog 1 --drive-pacer-burst 1 --drive-pacer-min-sleep 1s $(local_file) $(remote_dir)`)
#             success = true
#             # sleep(delay)
#         catch e
#             try
#                 println("Error uploading\n$(local_file)\n$(e)\nRetrying on my API keys.")
#                 remote_file = replace(local_file, "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/" => "snyder-virome-alt:")
#                 remote_dir = dirname(remote_file)
#                 run(`/scg/apps/software/rclone/1.67.0/rclone copy --drive-chunk-size 1G --drive-upload-cutoff 1T --tpslimit 1 --tpslimit-burst 0 --low-level-retries=1 --retries=1 --retries-sleep=1m --checkers 1 --transfers 1 --max-backlog 1 --drive-pacer-burst 1 --drive-pacer-min-sleep 1s $(local_file) $(remote_dir)`)
#                 success = true
#             catch e2
#                 println("Error uploading\n$(local_file)\n$(e2)\nRetrying in $(delay * (2 ^ attempt)) seconds.")
#                 sleep(delay * (2 ^ attempt))
#                 attempt += 1
#             end
#         end
#     end
#     if success
#         # Delete local file
#         println("Successfuly uploaded $(local_file). removing...")
#         rm(local_file)
#     else
#         println("Failed to upload $(local_file) after $(attempt) attempts.")
#         println("Try again later?")
#         break
#     end
# end